In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
train['lgb_pred'] = np.load("../input/30d-ml-2021-lgb-no-pseudo/lgb_oof.npy")
test['lgb_pred'] = np.load("../input/30d-ml-2021-lgb-no-pseudo/lgb_pred.npy")

train['xgb_pred'] = np.load("../input/30d-ml-2021-xgb-no-pseudo/xgb_oof.npy")
test['xgb_pred'] = np.load("../input/30d-ml-2021-xgb-no-pseudo/xgb_pred.npy")

train['lgb2_pred'] = np.load("../input/30d-ml-2021-lgb-model-2-no-pseudo/lgb_oof.npy")
test['lgb2_pred'] = np.load("../input/30d-ml-2021-lgb-model-2-no-pseudo/lgb_pred.npy")

train['xgb2_pred'] = np.load("../input/30d-ml-2021-xgb-model-2-no-pseudo/xgb_oof.npy")
test['xgb2_pred'] = np.load("../input/30d-ml-2021-xgb-model-2-no-pseudo/xgb_pred.npy")

train['ctb_pred'] = np.load("../input/30d-ml-2021-catb-no-pseudo/ctb_oof.npy")
test['ctb_pred'] = np.load("../input/30d-ml-2021-catb-no-pseudo/ctb_pred.npy")

In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont9,cont10,cont11,cont12,cont13,lgb_pred,xgb_pred,lgb2_pred,xgb2_pred,ctb_pred
0,B,B,B,C,B,B,A,E,C,N,...,0.267559,0.237281,0.377873,0.322401,0.869850,8.414218,8.425334,8.452603,8.430861,8.420048
1,B,B,A,A,B,D,A,F,A,O,...,0.341439,0.906013,0.921701,0.261975,0.465083,8.376001,8.361334,8.349284,8.328455,8.375351
2,A,A,A,C,B,D,A,D,A,F,...,0.843531,0.748809,0.620126,0.541474,0.763846,8.161854,8.177517,8.209423,8.190731,8.143454
3,B,B,A,C,B,D,A,E,C,K,...,0.574844,0.346010,0.714610,0.540150,0.280682,8.391968,8.398763,8.421653,8.401320,8.357306
4,A,A,A,C,B,D,A,E,A,N,...,0.956692,1.000773,0.776742,0.625849,0.250823,8.236148,8.256151,8.253373,8.255340,8.248562


# Preprocessing

In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [8]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
                #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 0.001, 20.017),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 0.001, 12),
                'subsample': trial.suggest_uniform('subsample', 0.266, 0.666),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.655, 1.0),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 1e-2, 1e-1),
                 'min_child_samples':trial.suggest_int('min_child_samples', 2, 50),
              'num_leaves':trial.suggest_int('num_leaves', 3, 61),
              'max_depth':trial.suggest_int('max_depth', 2, 15),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 761, 1161),
              'cat_smooth':trial.suggest_int('cat_smooth', 25, 75),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 50)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        #categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-30 04:13:10,900] A new study created in memory with name: no-name-98daabd8-d389-4c6b-98e2-b9d21f7cdadb


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[188]	valid_0's rmse: 0.713633
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[180]	valid_0's rmse: 0.713669


[I 2021-08-30 04:13:34,088] Trial 0 finished with value: 0.7136363148572867 and parameters: {'reg_lambda': 1.8943967735776317, 'reg_alpha': 11.376724556823065, 'subsample': 0.5435511640940591, 'colsample_bytree': 0.6798170818696656, 'learning_rate': 0.020460369060154117, 'min_child_samples': 10, 'num_leaves': 52, 'max_depth': 7, 'max_bin': 940, 'cat_smooth': 62, 'cat_l2': 0.878704549413933}. Best is trial 0 with value: 0.7136363148572867.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[413]	valid_0's rmse: 0.713385
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[503]	valid_0's rmse: 0.713352


[I 2021-08-30 04:13:53,411] Trial 1 finished with value: 0.7133635397332245 and parameters: {'reg_lambda': 16.046197871080437, 'reg_alpha': 1.0263227722928279, 'subsample': 0.5007809773393735, 'colsample_bytree': 0.6902873773837921, 'learning_rate': 0.015834989490828035, 'min_child_samples': 42, 'num_leaves': 8, 'max_depth': 10, 'max_bin': 786, 'cat_smooth': 47, 'cat_l2': 23.64184938850266}. Best is trial 1 with value: 0.7133635397332245.


...

Early stopping, best iteration is:
[227]	valid_0's rmse: 0.71337
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[157]	valid_0's rmse: 0.713359
Training until validation scores don't improve for 200 rounds


[I 2021-08-30 04:19:20,895] Trial 28 finished with value: 0.7133569312016007 and parameters: {'reg_lambda': 0.09025370222103937, 'reg_alpha': 3.5839003817055177, 'subsample': 0.5464155265988371, 'colsample_bytree': 0.8593545236561634, 'learning_rate': 0.05641981452865167, 'min_child_samples': 39, 'num_leaves': 10, 'max_depth': 6, 'max_bin': 1161, 'cat_smooth': 27, 'cat_l2': 0.0065544206051565855}. Best is trial 12 with value: 0.7133268070951851.


Early stopping, best iteration is:
[155]	valid_0's rmse: 0.713378
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[222]	valid_0's rmse: 0.713405
Training until validation scores don't improve for 200 rounds


[I 2021-08-30 04:19:36,880] Trial 29 finished with value: 0.7134055616911394 and parameters: {'reg_lambda': 1.7319626916115283, 'reg_alpha': 1.562230299058049, 'subsample': 0.531862911504736, 'colsample_bytree': 0.9272556496765181, 'learning_rate': 0.02405574710353151, 'min_child_samples': 5, 'num_leaves': 29, 'max_depth': 4, 'max_bin': 929, 'cat_smooth': 60, 'cat_l2': 0.06841793291159051}. Best is trial 12 with value: 0.7133268070951851.


Early stopping, best iteration is:
[181]	valid_0's rmse: 0.713422
Number of finished trials: 30
Best trial: {'reg_lambda': 0.8690000919820293, 'reg_alpha': 0.2988346804095643, 'subsample': 0.6566525907452626, 'colsample_bytree': 0.8834525807462618, 'learning_rate': 0.04837481160786926, 'min_child_samples': 50, 'num_leaves': 20, 'max_depth': 2, 'max_bin': 1147, 'cat_smooth': 73, 'cat_l2': 0.024808123536732993}


In [14]:
study.best_params

{'reg_lambda': 0.8690000919820293,
 'reg_alpha': 0.2988346804095643,
 'subsample': 0.6566525907452626,
 'colsample_bytree': 0.8834525807462618,
 'learning_rate': 0.04837481160786926,
 'min_child_samples': 50,
 'num_leaves': 20,
 'max_depth': 2,
 'max_bin': 1147,
 'cat_smooth': 73,
 'cat_l2': 0.024808123536732993}

# Log

======= 5 preds ==========
====== Ordinal encoding =========

0.7133926867644429 no noise ver14

====== One-hot encoding =========

0.7133625023983746 no noise ver13

wait nar

======= 4 preds ==========

====== Ordinal encoding =========

0.7137357370735623 no noise ver1

====== One-hot encoding =========

0.7137291748487112 no noise ver2

0.7137002996077744 no noise ver3 (narrow space) final

======= 3 preds ==========

0.7137535191229942 no noise ver5

0.713726763788908 no noise ver6 (narrow space)

0.7137241864554145 no noise ver7 (narrow space)

0.7137232072697571 no noise ver8 (narrow space)

0.7137179037352981 no noise ver9 (narrow space) final